In [1]:
# ebbbe00f-2b98-441b-92c0-1d820f987d8a
# Simple PySpark Script to Get Gateway Role Assignments from Microsoft Fabric API
# ------------------------------------------------------------------------------

# Import required libraries
import requests
import json
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.sql import SparkSession
from notebookutils.mssparkutils.credentials import getToken

# Initialize Spark Session - this is already available in Fabric Notebooks
spark = SparkSession.builder.appName("FabricGatewayRoleAssignments").getOrCreate()

# ===== ENTER YOUR GATEWAY ID BELOW =====
# Replace the text between the quotes with your actual gateway ID
gateway_id = "ebbbe00f-2b98-441b-92c0-1d820f987d8a"  # e.g. "a1b2c3d4-5678-90ab-cdef-123456789012"
# =======================================

def get_gateway_role_assignments(gateway_id):
    """
    Function to get gateway role assignments using the Fabric API with built-in authentication
    
    Parameters:
    gateway_id (str): The ID of the gateway you want to check
    
    Returns:
    DataFrame: A Spark DataFrame containing the gateway role assignments
    """
    # Get access token using built-in Fabric authentication
    try:
        # This gets a token for the Microsoft Fabric resource
        print("Getting access token using built-in authentication...")
        access_token = getToken("https://api.fabric.microsoft.com")
        
        # Set up API endpoint URL - no need for tenant ID as it's handled by the authentication
        api_url = f"https://api.fabric.microsoft.com/v1/gateways/{gateway_id}/roleAssignments"
        
        # Set up headers with authentication token
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }
        
        # Make the API request
        print("Sending API request to get gateway role assignments...")
        response = requests.get(api_url, headers=headers)
        
        # Check if request was successful
        if response.status_code == 200:
            print("API request successful!")
            
            # Parse the JSON response
            data = response.json()
            
            # Print the raw JSON data for inspection (pretty-printed)
            print("\nRaw API Response (first 5000 characters):")
            print(json.dumps(data, indent=2)[:5000] + "...\n")
            
            # Convert the JSON response to a Spark DataFrame
            # First, create a Python list from the JSON data
            role_assignments = data.get("value", [])
            
            if not role_assignments:
                print("No role assignments found for this gateway.")
                return spark.createDataFrame([], StructType([]))
            
            # Create a Spark DataFrame from the Python list
            role_df = spark.createDataFrame(role_assignments)
            
            print(f"Successfully retrieved {role_df.count()} gateway role assignments.")
            return role_df
            
        else:
            print(f"Error: API request failed with status code {response.status_code}")
            print(f"Response content: {response.text}")
            return None
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Run the function with the gateway_id provided above
print(f"Retrieving role assignments for gateway ID: {gateway_id}")
role_assignments_df = get_gateway_role_assignments(gateway_id)

# Display the results
if role_assignments_df is not None:
    print("\nGateway Role Assignments Preview:")
    role_assignments_df.show(100, truncate=False)  # Show first 10 rows, don't truncate columns
    
    print("\nDataFrame Schema:")
    role_assignments_df.printSchema()  # Display the schema of the DataFrame

StatementMeta(, 3a34127c-da99-461c-b085-a5557cc8147d, 3, Finished, Available, Finished)

Retrieving role assignments for gateway ID: ebbbe00f-2b98-441b-92c0-1d820f987d8a
Getting access token using built-in authentication...
Sending API request to get gateway role assignments...
API request successful!

Raw API Response (first 5000 characters):
{
  "value": [
    {
      "id": "2ebde336-f389-4b45-b24c-9bb881412734",
      "principal": {
        "id": "2ebde336-f389-4b45-b24c-9bb881412734",
        "type": "User"
      },
      "role": "Admin"
    },
    {
      "id": "c0fc694a-3df5-42b7-abba-d89a52a40086",
      "principal": {
        "id": "c0fc694a-3df5-42b7-abba-d89a52a40086",
        "type": "User"
      },
      "role": "Admin"
    },
    {
      "id": "d87d15b5-62cd-41e0-87fd-92329908fadb",
      "principal": {
        "id": "d87d15b5-62cd-41e0-87fd-92329908fadb",
        "type": "User"
      },
      "role": "Admin"
    },
    {
      "id": "f12096e4-4ee8-4adc-8f64-1b25cc463fc4",
      "principal": {
        "id": "f12096e4-4ee8-4adc-8f64-1b25cc463fc4",
        "type